In [1]:
%%capture
!pip3 install penngrader --upgrade
!pip3 install PyYAML

In [6]:
import json
import dill
import base64
import types
import ast
import types
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime
import inspect
from difflib import SequenceMatcher

from urllib.error import HTTPError

# Request types
HOMEWORK_ID_REQUEST     = 'GET_HOMEWORK_ID'
UPDATE_METADATA_REQUEST = 'UPDATE_METADATA'
UPDATE_TESTS_REQUEST    = 'UPDATE_TESTS'

def is_function(val):
    return inspect.isfunction(val)

def is_module(val):
    return inspect.ismodule(val)

def is_class(val):
    return inspect.isclass(val)

def is_external(name):
    return name not in ['__builtin__','__builtins__', 'penngrader','_sh', '__main__'] and 'penngrader' not in name


class PennGraderBackend:
    
    def __init__(self, secret_key, homework_number):
        self.secret_key = secret_key
        self.homework_number = homework_number
        self.homework_id = self._get_homework_id()
        if 'Error' not in self.homework_id:
            response  = 'Success! Teacher backend initialized.\n\n'
            response += 'Homework ID: {}'.format(self.homework_id)
            print(response)
        else:
            print(self.homework_id)
            
    def update_metadata(self, deadline, total_score, max_daily_submissions):
        request = { 
            'homework_number' : self.homework_number, 
            'secret_key' : self.secret_key, 
            'request_type' : UPDATE_METADATA_REQUEST,
            'payload' : self._serialize({
                'max_daily_submissions' : max_daily_submissions,
                'total_score' : total_score,
                'deadline' : deadline
            })
        }
        print(self._send_request(request, config_api_url, config_api_key))
    
            
    def update_test_cases(self):
        request = { 
            'homework_number' : self.homework_number, 
            'secret_key' : self.secret_key, 
            'request_type' : UPDATE_TESTS_REQUEST,
            'payload' : self._serialize({
                'libraries'  : self._get_imported_libraries(),
                'test_cases' : self._get_test_cases(),
            })
        }
        print(self._send_request(request, config_api_url, config_api_key))
    
    
    def _get_homework_id(self):
        request = { 
            'homework_number' : self.homework_number,
            'secret_key' : self.secret_key,
            'request_type' : HOMEWORK_ID_REQUEST,
            'payload' : self._serialize(None)
        }
        return self._send_request(request, config_api_url, config_api_key)

        
    def _send_request(self, request, api_url, api_key):
        params = json.dumps(request).encode('utf-8')
        headers = {'content-type': 'application/json', 'x-api-key': api_key}
        try:
          request = urllib.request.Request(api_url, data=params, headers=headers)
        except err:
          return 'Request builder error: {}'.format(err.read().decode("utf-8")) 
        try:
            response = urllib.request.urlopen(request)
            return '{}'.format(response.read().decode('utf-8'))
        except HTTPError as error:
            return 'Http Error: {}'.format(error.read().decode("utf-8")) 
        
    
    def _get_imported_libraries(self):
        # Get all externally imported base packages
        packages = set() # (package, shortname)
        for shortname, val in list(globals().items()):
            if is_module(val) and is_external(shortname):
                base_package = val.__name__.split('.')[0]
                if base_package != 'google' and base_package != 'yaml':
                  packages.add(base_package)
            if (is_function(val) or is_class(val)) and is_external(val.__module__):
                base_package = val.__module__.split('.')[0]
                packages.add(base_package)
        print ('Importing packages ', packages)

        # Get all sub-imports i.e import sklearn.svm etc 
        imports = set() # (module path , shortname )
        for shortname, val in list(globals().items()):
            if is_module(val) and is_external(shortname):
                if val.__name__ in packages:
                    packages.remove(val.__name__)
                if shortname != 'drive' and shortname != 'yaml':
                  imports.add((val.__name__, shortname))

        print ('Importing libraries ', imports)
        # Get all function imports 
        functions = set() # (module path , function name)
        for shortname, val in list(globals().items()):
            if is_function(val)and is_external(val.__module__):
                functions.add((val.__module__, shortname))     
        print ('Importing functions ', functions)

        return {
            'packages' : list(packages), 
            'imports' : list(imports), 
            'functions' : list(functions)
        }

    
    def _get_test_cases(self):
        # Get all function imports 
        test_cases = {}
        for shortname, val in list(globals().items()):
            try:
                if val and is_function(val) and not is_external(val.__module__) and \
                'penngrader' not in val.__module__:
                  test_cases[shortname] = inspect.getsource(val)   
                  print ('Adding case {}', shortname)
            except:
                print ('Skipping {}', shortname)
                pass
        return test_cases

    
    def _serialize(self, obj):
        '''Dill serializes Python object into a UTF-8 string'''
        byte_serialized = dill.dumps(obj, recurse = False)
        return base64.b64encode(byte_serialized).decode("utf-8")

    
    def _deserialize(self, obj):
        byte_decoded = base64.b64decode(obj)
        return dill.loads(byte_decoded)

# PennGrader: Teacher Backend

Follow this notebook's insutructions to write your homework's test cases and see student's grades.
### Configuration
Edit the following variables and run the following cell to initialize the teacher backend.

`SECRET_KEY`: The secret key you obtained when you created your course 

`HOMEWORK_NUMBER`: Homework number you are writing tests for. 

(Please do not edit the metadata of the current homeowork without changning the SECRET_KEY, this is just a template/demo)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import yaml
with open(r"/content/drive/config.yaml") as config_file:
    student_ids = []
    config = yaml.load(config_file)

    config_api_url = config['config_api_url']
    config_api_key = config['config_api_key']

    SECRET_KEY      = config['secret_id']

    # Change this for later homeworks
    HOMEWORK_NUMBER = 1

In [8]:
backend = PennGraderBackend(secret_key = SECRET_KEY, homework_number = HOMEWORK_NUMBER)

Success! Teacher backend initialized.

Homework ID: CIS545_Spring_2019_HW1


### Metadata
Edit the following metadata variables to your preference:

`TOTAL_SCORE`: Total number of points this homework is worth. **Note:** Make sure all test cases weigths add up to this number.

`DEADLINE`:... you guessed it, the homework deadline. (_Format:_ `'YYYY-MM-DD HH:MM A'`)

`MAX_DAILY_TEST_CASE_SUBMISSIONS`: Maximum number of daily submissions per test case per student.

In [9]:
TOTAL_SCORE = 20
DEADLINE = '2019-12-05 11:59 PM'
MAX_DAILY_TEST_CASE_SUBMISSIONS = 100

In [10]:
backend.update_metadata(DEADLINE, TOTAL_SCORE, MAX_DAILY_TEST_CASE_SUBMISSIONS)

Success! Metadata updated.

Total HW Points: 20
Deadline: 2019-12-05 23:59:00
Max daily submissions per test case: 100



### Test Cases
Define a test case function for each question.

A test case function takes in a single input containing the student's answer and returns a tuple `(student score:int, maximum score:int)`. See example below:


In [11]:
import numpy as np
import pandas as pd

def test_case_1(answer): 
    # [answer] in this test case is a function that adds 2 numbers
    
    # First initalize the max_score score of this test case. 
    max_score     = 5
    student_score = 0

    # Since answer is a function that takes two parameters 
    # answer(1,2) should return 3 if implemented correctly.
    if answer(1,2) == 3:
        student_score += 3
    if answer(2,2) == 4:
        student_score += 2
    
    # Returning a (student_score, max_score) tuple of ints
    return (student_score, max_score)

def test_case_2(answer): 
    max_score     = 10
    student_score = 0
    
    students_df = answer 
    
    students_df = students_df.sort_values('first_name', ascending = False)
    if students_df.iloc[0].first_name == 'Leonardo':
        student_score += 10
        
    return (student_score, max_score)

Run the following cell to update the test cases.

In [12]:
backend.update_test_cases()

Success: Test cases updated successfully.


 
 

 ### View Grades

Run the following cells to view student's scores.
 
 
     

In [13]:
grades_df = backend.get_grades()
grades_df

,student_id,student_score,latest_submission,deadline,days_late
0,99999999,20,2019-12-18 17:58:00.000005,2019-12-05 23:59:00,13


Run the following cells to view the raw student's scores for each test case.

In [14]:
raw_grades_df = backend.get_raw_grades() # Note: timestamp is in UTC
raw_grades_df

,max_score,homework_id,student_submission_id,student_score,timestamp
0,5,CIS545_Spring_2019_HW1,99999999_test_case_1,5,2019-12-18 22:57
1,5,CIS545_Spring_2019_HW1,99999999_test_case_2,5,2019-12-18 22:58
2,10,CIS545_Spring_2019_HW1,99999999_test_case_3,10,2019-12-18 22:58
